In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import json
import requests
import numpy as np
from pandas.io.json import json_normalize
import os
import time

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [3]:
def get_api_result(point,date_start,date_end):
  point = str(point)
  url  ='http://110.164.131.86:9062/forecast_data/export?point='+ point + '&release_start='+date_start+'T00:00:00&release_end='+date_end + 'T23:59:59'
  r = requests.get(url)
  if r.status_code == 200:
    if len(json.loads(r.content)['data'])==0:
      print('point '+str(point)+' Data is not found')
    return json.loads(r.content)
  else:
    print('Can not connect')
    
def J_to_pd(r): 
    if int(r['data'][0]['point'])%2 == 0:
      name = 'i^_' + str(r['data'][0]['site'])
    if int(r['data'][0]['point'])%2 == 1:
      name = 'p^_'+str(r['data'][0]['site']) 
    df = pd.json_normalize(r['data']) 
    df = df.set_index('forecast_time')
    df.index = pd.to_datetime(df.index, dayfirst=True)
    df['forecast_value']=df['forecast_value'].astype('float')
    df = df.rename(columns={'forecast_value': name})
    df.index.name = 'Datetime'
    df = df.drop(columns=['forecast_release_time','point','site'])
    return df

In [4]:
MEASURE_FOLDER = 'C:\\Users\\Tee\\senior_project\\src\\SolarMap\\DataAndResult\\measuredata' # Folder of irradiance 56 stations that mean 56 csv files (or greater)
META_FOLDER = 'C:\\Users\\Tee\\senior_project\\src\\SolarMap\\DataAndResult\\metadata'
measure_files = os.listdir(MEASURE_FOLDER)[1:]
measure_files.sort()

# Metadata creation (run only one time)

In [59]:
#plant_tiff_index =  pd.read_csv(os.path.join(META_FOLDER,'plant_metadata_tiff_index.csv'))
#plant_tiff_index

,site,lat,lng,CapacityDC_kW,CapacityAC_kW,row,col
0,ISL001,14.00552,100.5194,996.48,875.0,832,863
1,ISL002,13.70650,100.2812,199.88,175.0,849,850
2,ISL003,13.65380,100.5223,992.25,875.0,852,864
3,ISL004,13.63543,100.6322,425.25,375.0,853,870
4,ISL005,13.69280,100.8972,992.25,875.0,849,884
...,...,...,...,...,...,...,...
58,SPP2,15.36391,100.3117,NaN,126126.0,754,852
59,SPP3,17.07856,100.1173,NaN,133920.0,655,841
60,SPP4,15.05355,100.8931,NaN,73000.0,772,884
61,SPP5,15.01876,100.7042,NaN,52000.0,774,874


In [6]:
#url = 'http://110.164.131.86:9062/sites/all'
#r = requests.get(url)
#database = [json.loads(r.content)]
#database

[[{'url': 'http://110.164.131.86:9062/sites/1/',
   'location_id': 1,
   'site_name': '001',
   'site_type': 'Solar Rooftop',
   'site_capacity': 875,
   'latitude': '14.0052300000',
   'longitude': '100.5194030000',
   'owner': 1,
   'access_level': 'NDA.ISL.20190821.2Years',
   'acquisition_method': 'Pull Proprietary API, daily request at 1:00am',
   'acquisition_parameter': 'https://api.impactsolar.co.th:8080/meterLocationEnergyData/001',
   'points': [1, 2, 3, 253, 1279, 1280],
   'site_enable': True,
   'date_available': '2018-12-30T00:00:00+07:00',
   'create_date': '2019-09-20T09:37:27.288303+07:00',
   'modify_date': '2022-08-05T10:14:21.265437+07:00',
   'remark': '{"Sensor":Y}',
   'tile_id': '13220313233310'},
  {'url': 'http://110.164.131.86:9062/sites/9/',
   'location_id': 9,
   'site_name': '002',
   'site_type': 'Solar Rooftop',
   'site_capacity': 175,
   'latitude': '13.7065000000',
   'longitude': '100.2812000000',
   'owner': 1,
   'access_level': 'NDA.ISL.20190821.

In [54]:
#metadata = []
#for e in database[0][:56]:
#    metadata.append({'site_name':'ISL'+e['site_name'],'location_id':e['location_id'],'site_capacity':float(e['site_capacity']),\
#                     'point_p':str(e['points'][-2]),'point_i':str(e['points'][-1]),'lat':e['latitude'], 'lng':e['longitude'],'tile_id':e['tile_id']})
#for e in database[0][57:58]:
#    metadata.append({'site_name':e['site_name'],'location_id':e['location_id'],'site_capacity':float(e['site_capacity']),\
#                      'lat':e['latitude'], 'lng':e['longitude'],'tile_id':e['tile_id']})
#for e in database[0][562:568]:
#      metadata.append({'site_name':e['site_name'],'location_id':e['location_id'],'site_capacity':float(e['site_capacity']),\
#                        'point_p':str(e['points'][-2]),'point_i':str(e['points'][-1]),'lat':e['latitude'], 'lng':e['longitude'],'tile_id':e['tile_id']})

In [62]:
# SRTM30m dataset
#elev = []
#for i in range(len(plant_df.index)):
#    url  ='https://api.opentopodata.org/v1/srtm30m?locations='+str(plant_df.lat.values[i])+','+str(plant_df.lng.values[i])
#    r = requests.get(url)
#    elev.append(r.json()['results'][0]['elevation'])
#    time.sleep(1)

In [63]:
#plant_df = pd.DataFrame(metadata)
#plant_df['alt'] = elev
#plant_df['row'] = plant_tiff_index['row']
#plant_df['col'] = plant_tiff_index['col']

In [67]:
#plant_df.to_csv('plant_metadata.csv')

In [5]:
metadata_df =  pd.read_csv(os.path.join(META_FOLDER,'plant_metadata.csv'))

# estimation irradiance API requesting

Model v1 period 2020-11-xx - 2022-09-06  
Model v2 period 2022-09-06 - 2023-04-30

In [5]:
date_start = '2020-10-01'
date_end =  '2022-09-05'
id2locate = {}
for e in metadata_df.location_id.values[:56]:
    id2locate[e] = metadata_df[metadata_df.location_id == e].site_name.values[0]

i_data=[]
for e in metadata_df.point_i.values[:56]:
    r =get_api_result(str(int(e)),date_start,date_end)
    r = r['data']
    for e in r:
        i_data.append({'datetime':e['forecast_time'],'site_name':id2locate[e['site']],'ihat_wpm2':float(e['forecast_value'])})

point 1324 Data is not found
point 1330 Data is not found
point 1336 Data is not found
point 1342 Data is not found
point 1348 Data is not found
point 1354 Data is not found
point 1360 Data is not found
point 1366 Data is not found
point 1372 Data is not found
point 1378 Data is not found
point 1384 Data is not found
point 1390 Data is not found
point 1396 Data is not found
point 1402 Data is not found
point 1408 Data is not found
point 1414 Data is not found
point 1420 Data is not found
point 1426 Data is not found
point 1432 Data is not found
point 1438 Data is not found
point 1444 Data is not found
point 1450 Data is not found
point 1456 Data is not found
point 1462 Data is not found
point 1468 Data is not found
point 1474 Data is not found
point 1480 Data is not found
point 1486 Data is not found
point 1492 Data is not found
point 1498 Data is not found
point 1504 Data is not found
point 1510 Data is not found
point 1516 Data is not found
point 1522 Data is not found
point 1528 Dat

In [6]:
ihatdf=pd.DataFrame(i_data)

In [7]:
q=ihatdf.copy()
w= q.groupby(by=['site_name']).mean()[['ihat_wpm2']]
w['N'] = q.groupby(by=['site_name']).count()['ihat_wpm2']
w

,ihat_wpm2,N
site_name,,
ISL001,208.705940,26705
ISL002,192.377007,26700
ISL003,196.904337,26700
ISL004,209.935068,26701
ISL005,207.496105,26699
ISL006,198.003549,26703
ISL007,211.278261,26708
ISL008,216.699494,26704
ISL009,215.574729,26714


In [11]:
q = ihatdf.set_index('datetime')
q.index =pd.to_datetime(q.index).tz_convert('Asia/Bangkok')
q = q.between_time('6:00','18:00')
q.index.name = 'Datetime'
q.to_csv('ihat_only_modelv1.csv')

In [13]:
date_start = '2022-09-06'
date_end =  '2023-04-30'
id2locate = {}
for e in metadata_df.location_id.values[:56]:
    id2locate[e] = metadata_df[metadata_df.location_id == e].site_name.values[0]

i_data=[]
for e in metadata_df.point_i.values[:56]:
    r =get_api_result(str(int(e)),date_start,date_end)
    r = r['data']
    for e in r:
        i_data.append({'datetime':e['forecast_time'],'site_name':id2locate[e['site']],'ihat_wpm2':float(e['forecast_value'])})

In [14]:
ihatdf=pd.DataFrame(i_data)

In [15]:
q=ihatdf.copy()
w= q.groupby(by=['site_name']).mean()[['ihat_wpm2']]
w['N'] = q.groupby(by=['site_name']).count()['ihat_wpm2']
w

,ihat_wpm2,N
site_name,,
ISL001,430.572811,5132
ISL002,422.679420,5132
ISL003,430.572400,5132
ISL004,432.930639,5132
ISL005,432.397558,5132
ISL006,429.884672,5132
ISL007,439.840929,5132
ISL008,430.594751,5132
ISL009,446.741797,5132


In [16]:
q = ihatdf.set_index('datetime')
q.index =pd.to_datetime(q.index).tz_convert('Asia/Bangkok')
q = q.between_time('6:00','18:00')
q.index.name = 'Datetime'
q.to_csv('ihat_only_modelv2.csv')

# Aggregration measurement irradiance

In [15]:
imsdf=pd.DataFrame()

for i in range(len(measure_files)):
    idf=pd.DataFrame()
    df = pd.read_csv(os.path.join(MEASURE_FOLDER,measure_files[i]),parse_dates=['Datetime'],index_col='Datetime')
    df = df[(df.index.minute==0)|(df.index.minute==30)]
    df = df.between_time('6:00','18:00')
    idf['site_name']= '';
    idf['ims_wpm2']=df[df.columns[2]]
    idf['site_name']= 'ISL'+measure_files[i].split('.')[0].split('-')[0]
    imsdf=pd.concat([imsdf,idf],axis=0,sort=False)
    
imsdf.index = imsdf.index.tz_localize('Asia/Bangkok')

In [17]:
imsdf.reset_index(inplace=True)
imsdf.drop_duplicates(inplace=True)
imsdf.dropna(inplace=True)

In [19]:
imsdf.to_csv('ims.csv')